In [ ]:
fig, ax = plt.subplots()
ax, cdf1 = gf.graph_cdf(only_child_group["rating"], ax=ax, label="Only Child", color='orange', show=False)
ax, cdf2 = gf.graph_cdf(not_only_child_group["rating"], ax=ax, label="Not Only Child", color='green', show=False)
gf.find_point_largest_gap(cdf1, cdf2)

# only include 0-4 ratings for gap analysis
cdf1 = cdf1[:]
cdf2 = cdf2[:]
gap_start_x, gap_end_x = gf.find_point_largest_gap(cdf1, cdf2)

# find the y-value for only child for annotation
only_child_y = cdf1[gap_end_x]
not_only_child_y = cdf2[gap_end_x]

# place back to rating scale since it is 0.5 increments
rating = [i * 0.5 for i in range(0, 9)]
gap_start_x = rating[gap_start_x]
gap_end_x = rating[gap_end_x]


ax.legend()

plt.show()
plt.close()



In [ ]:
# Calculate the differeces between the two CDFs at each rating and plot them 
gf.plot_absolute_differences(rating, cdf1, cdf2)